### 추가 참고 : Automated Machine Learning with Auto-Keras @ http://j.mp/2xeG2c2

#### A. Keras library 를 활용한 방법

In [29]:
import tensorflow as tf
from tensorflow.keras import datasets, models, layers, utils, losses # tf.keras 에 필요한 함수들이 모여있습니다.

In [30]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # https://stackoverflow.com/questions/35911252/disable-tensorflow-debugging-information

<br> 

#### 1. Prepare train & test data (MNIST)

#### Train & Test split

In [31]:
(train_data, train_label), (test_data, test_label) = datasets.mnist.load_data()

print(train_data.shape) # # of training data == 60000, each data = 28px * 28px
print(test_data.shape) # # of test data == 10000
print(train_data.dtype) # int
print(train_data.max()) # 0~255

(60000, 28, 28)
(10000, 28, 28)
uint8
255


In [32]:
train_label

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

#### Normalization

In [33]:
train_data = train_data.reshape(60000, 784).astype('float32') / 255.0 # 0~255 -> 0~1
test_data = test_data.reshape(10000, 784).astype('float32') / 255.0 # 0~255 -> 0~1

#### One-hot encoding - 데이터 중복제거를 위해 벡터로 학습

In [34]:
train_label

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [35]:
train_label = utils.to_categorical(train_label) # 0~9 -> one-hot vector
test_label = utils.to_categorical(test_label) # 0~9 -> one-hot vector

In [36]:
train_label

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

<br> 

#### 2. Build the model & Set the criterion 

Sequential vs Functional

In [37]:
model = models.Sequential() # Build up the "Sequence" of layers (Linear stack of layers)

In [38]:
model.add(layers.Dense(input_dim=28*28,
                       units=512, #퍼셉트론 수
                       activation='relu',
                       kernel_initializer='he_uniform')) # Dense-layer (relu & he)
model.add(layers.Dropout(0.2)) # Dropout-layer
model.add(layers.Dense(units=10, activation='softmax')) # (Output) Dense-layer with softmax function, 0~9 -> 10

In [39]:
# "Compile" the model description (Configures the model for training)
model.compile(optimizer='adam', 
              loss=losses.categorical_crossentropy, # See other available losses @ https://keras.io/losses/
              metrics=['accuracy'])

<br> 

#### 3. Train the model

In [44]:
# Fit the model on training data

model.fit(train_data, train_label, batch_size=100, epochs=10) # default batch_size = 32

Epoch 1/10
60000/60000 [==============================] - 3s 43us/sample - loss: 0.0205 - acc: 0.9934
Epoch 2/10
60000/60000 [==============================] - 3s 42us/sample - loss: 0.0147 - acc: 0.9952
Epoch 3/10
60000/60000 [==============================] - 3s 42us/sample - loss: 0.0142 - acc: 0.9952
Epoch 4/10
60000/60000 [==============================] - 3s 42us/sample - loss: 0.0125 - acc: 0.9960
Epoch 5/10
60000/60000 [==============================] - 3s 42us/sample - loss: 0.0110 - acc: 0.9964
Epoch 6/10
60000/60000 [==============================] - 3s 42us/sample - loss: 0.0102 - acc: 0.9967
Epoch 7/10
60000/60000 [==============================] - 3s 42us/sample - loss: 0.0105 - acc: 0.9964
Epoch 8/10
60000/60000 [==============================] - 2s 41us/sample - loss: 0.0100 - acc: 0.9966
Epoch 9/10
60000/60000 [==============================] - 2s 41us/sample - loss: 0.0106 - acc: 0.9966
Epoch 10/10
60000/60000 [==============================] - 2s 42us/sample - loss: 

<br> 

#### 4. Test the model

In [45]:
# Evaluate the model on test data

result = model.evaluate(test_data, test_label, batch_size=100)

10000/10000 [==============================] - 0s 25us/sample - loss: 0.0507 - acc: 0.9866


In [46]:
print('loss (cross-entropy) :', result[0])
print('test accuracy :', result[1])

loss (cross-entropy) : 0.0506739641958211
test accuracy : 0.9866


<br>

#### B. Tensorflow 의 keras API 를 활용한 방법 (@ https://goo.gl/v8h2SH)

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
# or
# x_train = tf.keras.utils.normalize(x_train, axis=1)  # scales data between 0 and 1
# x_test = tf.keras.utils.normalize(x_test, axis=1)  # scales data between 0 and 1

model = tf.keras.models.Sequential()  # a basic feed-forward model
model.add(tf.keras.layers.Flatten())  # takes our 28x28 and makes it 1x784
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))  # a simple fully-connected layer, 512 units, relu activation
model.add(tf.keras.layers.Dropout(0.2))  
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))  # our output layer. 10 units for 10 classes. Softmax for probability distribution
# or
# model = tf.keras.models.Sequential([
#   tf.keras.layers.Flatten(),
#   tf.keras.layers.Dense(512, activation=tf.nn.relu),
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.Dense(10, activation=tf.nn.softmax)
# ])

# if labels are not one-hot vectors, we should use "sparse_categorical_crossentropy" (http://j.mp/2XS0jmv)
model.compile(optimizer='adam',  # Good default optimizer to start with
              loss=losses.sparse_categorical_crossentropy,  # how will we calculate our "error." Neural network aims to minimize loss.
              metrics=['accuracy'])  # what to track

model.fit(x_train, y_train, epochs=5)

val_loss, val_acc = model.evaluate(x_test, y_test)  # evaluate the out of sample data with model
print('loss (cross-entropy) :', val_loss)
print('test accuracy :', val_acc)

# import numpy as np
# import matplotlib.pyplot as plt
# predictions = model.predict(x_test)
# print(np.argmax(predictions[0]))
# plt.imshow(x_test[0],cmap=plt.cm.binary)
# plt.show()